In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field

load_dotenv()


True

In [4]:
# 定义系统提示和用户提示
system_prompt = """You are the AWS Lambda expert. You have to generate AWS Lambda codes with Python 3.11. 
When I ask you question, you have to provide 1. AWS Lambda codes and 2. requirements.txt which includes all necessary python libraries.
Output format is :
------------------------------
{{AWS Lambda codes}}
------------------------------
{{requirements.txt}}
------------------------------
"""

user_prompt = """Please grab the stock market opening price of the previous trading day according to TAIEX URL: 
https://www.twse.com.tw/indicesReport/MI_5MINS_HIST?response=json"""

class LambdaCodes(BaseModel):
    codes: str = Field(description="AWS Lambda codes that can solve user's questions.")
    requirements_txt: str = Field(description="requirements.txt which includes all necessary python libraries use in AWS lambda codes.")

In [5]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=LambdaCodes)

prompt = PromptTemplate(
    template="Answer the user question.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

chain.invoke({"query": user_prompt})

{'codes': "import requests\n\nurl = 'https://www.twse.com.tw/indicesReport/MI_5MINS_HIST?response=json'\nresponse = requests.get(url)\ndata = response.json()\nopening_price = data['data'][0]['z']\nprint(opening_price)",
 'requirements_txt': 'requests'}

In [10]:
output_parser

StrOutputParser()